In [1]:
import pandas as pd
import numpy as np
import duckdb

In [4]:
# 행 개수 설정
n_rows = 10000

# 1. 카테고리 데이터 생성
categories = ['Electronics', 'Clothing', 'Furniture', 'Books', 'Toys']
# 2. 판매처 데이터 생성
stores = ['Seoul', 'Busan', 'Daegu', 'Incheon']

data = {
    'Date': pd.date_range(start='2025-01-01', periods=n_rows, freq='h'), # 시간순 데이터
    'Category': np.random.choice(categories, n_rows), # 5개 카테고리 랜덤 할당
    'Store': np.random.choice(stores, n_rows),        # 4개 판매처 랜덤 할당
    'Sales': np.random.randint(10, 1000, n_rows),     # 판매량 (10~1000 사이)
    'Returned': np.random.choice([True, False], n_rows, p=[0.05, 0.95]) # 반품 여부 (5% 확률)
}

tbl = pd.DataFrame(data)

In [5]:
tbl.head()

,Date,Category,Store,Sales,Returned
0,2025-01-01 00:00:00,Electronics,Daegu,72,False
1,2025-01-01 01:00:00,Electronics,Seoul,829,False
2,2025-01-01 02:00:00,Clothing,Busan,74,False
3,2025-01-01 03:00:00,Books,Incheon,79,False
4,2025-01-01 04:00:00,Furniture,Incheon,715,False


In [6]:
tbl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      10000 non-null  datetime64[ns]
 1   Category  10000 non-null  object        
 2   Store     10000 non-null  object        
 3   Sales     10000 non-null  int32         
 4   Returned  10000 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int32(1), object(2)
memory usage: 283.3+ KB


In [15]:
duckdb.query('select Category, count(1) as CNT from tbl group by Category order by CNT').df()

,Category,CNT
0,Furniture,1979
1,Electronics,2000
2,Clothing,2003
3,Books,2006
4,Toys,2012


In [22]:
duckdb.query('''
select Category, Store, round(avg(Sales)) as AverageSales 
    from tbl 
    group by Category, Store
    order by Category ASC, AverageSales DESC
    ''').df()

,Category,Store,AverageSales
0,Books,Daegu,521.0
1,Books,Seoul,514.0
2,Books,Incheon,487.0
3,Books,Busan,477.0
4,Clothing,Busan,514.0
5,Clothing,Daegu,511.0
6,Clothing,Incheon,495.0
7,Clothing,Seoul,479.0
8,Electronics,Seoul,509.0
9,Electronics,Incheon,507.0


In [31]:
duckdb.query('''
select Category, sum(Sales) as TotalSales
    from tbl 
    where Returned = 'False'
    group by Category
    order by TotalSales DESC limit 3
    ''').df()

,Category,TotalSales
0,Toys,977606.0
1,Electronics,962266.0
2,Books,961888.0
